In [55]:
#imports

%matplotlib widget
%config InlineBackend.figure_format = 'svg'

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dts
import matplotlib.image as mpimg
import numpy as np

import ipywidgets as widgets 
from ipywidgets import interact, interact_manual, interactive, fixed, HBox, VBox, Label, Layout

import datetime as dt
from datetime import datetime
import time

from os import listdir

In [56]:
dirs = listdir('data_files/scan_data/multicoil')

#string format of dates
dateNums = [[] for i in range(len(dirs))]

mat = [[[] for j in range(9)] for i in range(len(dirs))]

#daily image
# dImg1 = mpimg.imread('data_files/daily_images/10deg_flip_angle/3te/birnfig1_fmri3te_new_10.jpg')
# dImg2 = mpimg.imread('data_files/daily_images/10deg_flip_angle/3te/birnfig2_fmri3te_new_10.jpg')
# dImg3 = mpimg.imread('data_files/daily_images/10deg_flip_angle/3te/birnfig3_fmri3te_new_10.jpg')
# dImg4 = mpimg.imread('data_files/daily_images/10deg_flip_angle/3te/birnfig4_fmri3te_new_10.jpg')

#parsing data from file
def parse():
    fc = 0
    for file in dirs:
        #data values (y-values)
        with open('data_files/scan_data/multicoil/'+file) as dataF:
            next(dataF)
            for line in dataF:
                data = line.split()
                #getting string form of date
                date = data[0]
                #reformatting date into readable format
                reformat = date[0:4]+'/'+date[4:6]+'/'+date[6:8]
                #print(reformat)
                dt = datetime.strptime(reformat, '%Y/%m/%d')
                dateNums[fc].append(str(dt))
                #important values
                mat[fc][0].append(dts.date2num(dt))
                for i in range(8):
                    mat[fc][i+1].append(float(data[i+1]))
                #if status becomes relevant/recorded
                #mat[fc][i].append(data[9])
        fc+=1

In [57]:
class Line:
    def __init__(self, fig, ax, dirs):
        self.fig = fig
        self.ax = ax
        self.annot = self.ax.annotate("", xy=(0,0), xytext=(-20, 20),textcoords="offset points",
                                                    bbox=dict(boxstyle="round", fc="w"),
                                                    arrowprops=dict(arrowstyle="->"),
                                                    fontfamily = 'Arial', fontsize = 9)
        self.annot.set_visible(False)
        self.dirs = dirs
        
    def draw_line(self, x, y, color):
        self.line, = self.ax.plot_date(x, y, '.')
        self.ax.plot_date(x, y, '.-', linewidth = 0.8, color=color)
        
    def update(self, ind, data_mat):
        x,y = self.line.get_data()
        x_i = ind["ind"][0]
        self.annot.xy = (x[ind["ind"][0]], y[ind["ind"][0]])
        text = "{}\n{}".format('Date: %s' % str(dts.num2date(data_mat[0][x_i]))[0:10],
                               'Value: %s' % str(data_mat[1][x_i])[0:4])
        self.annot.set_text(text)
        
    def on_hover(self, event, data_mat):
        if event.inaxes == self.ax:
            within, ind = self.line.contains(event)
            if within:
                self.annot.set_visible(True)
                self.update(ind, data_mat)
                self.fig.canvas.draw_idle()
            elif self.annot.get_visible():
                self.annot.set_visible(False)
                self.fig.canvas.draw_idle()
                
    def connect_comp(self, data_mat):
        self.fig.canvas.mpl_connect('motion_notify_event', 
                                    lambda event: self.on_hover(event, data_mat))
        #self.fig.canvas.mpl_connect('button_release_event',
                                    #lambda event: self.on_click(event, data))

In [58]:
class Axis:
    def __init__(self, fig, ax, dirs):
        self.fig = fig
        self.ax = ax
        self.ax.grid(True)
        self.dirs = dirs
        
    def draw_axes(self, data_mat, i):
        colors = ['#00008B', '#228B22', '#FF0000', '#40E0D0', '#800080', '#FFFF00', '#556B2F', '#89cff0']
        line = Line(self.fig, self.ax, self.dirs)
        line.draw_line(data_mat[0], data_mat[1], colors[i])
        
        line.connect_comp(data_mat)
                
    #def on_click(self, event, data):
        #print('click')

In [59]:
class Graph:
    def __init__(self, n, mat, val):
        self.init_date = 0
        self.n_points = n
        self.n = n
        self.mat = mat
        self.dirs = dirs
        self.val = val
        
        self.update_button = widgets.Button(
            value=False,
            description='Apply',
            disabled=False,
            button_style='',
            tooltip='Update Graph'
        )
        
        self.date_picker = widgets.Dropdown(
            description = 'Pick initial date:',
            disabled = False,
            style = {
                'description_width': 'initial'
            },
            options = [(dateNums[self.val][i][0:10], i) for i in range(len(self.mat[self.val][0]))]
        )
        
        self.n_slider = widgets.IntSlider(
            description = 'Number of Points:',
            value = min(self.n, len(self.mat[self.val][0])),
            min = 1,
            max = len(self.mat[self.val][0]),
            step = 1,
            continuous_update = False,
            style = {
                'description_width': 'initial'
            },
            layout = Layout(width = '381px')
        )
        
        display(self.date_picker)
        display(self.n_slider)
        display(self.update_button)
        self.update_button.on_click(self.button_click)
        
        self.fig, axs = plt.subplots(figsize=[10,9])
        plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.4)
        plt.rc('xtick', labelsize=8)
        
        self.ax_1 = Axis(self.fig, axs, self.dirs)
        
        self.draw()
        
    def draw(self):
        self.fig.suptitle('Multicoil')
        if self.val == 0:
            title = '3TE'
        elif self.val == 1:
            title = '3TW'
        else:
            title = 'Unknown'
        self.ax_1.ax.set_title(title)
        self.ax_1.ax.set_ylabel('Signal Intensity')
        
        for i in range(8):
            data_mat = []
            data_mat.append(self.mat[self.val][0][self.init_date:self.init_date+self.n_points])
            data_mat.append(self.mat[self.val][i+1][self.init_date:self.init_date+self.n_points])
            self.ax_1.draw_axes(data_mat, i)

        plt.setp(self.ax_1.ax.get_xticklabels(), rotation=30, ha="right")

        plt.show(block=False)

        #imgplot=plt.imshow(dImg1)
    
    def button_click(self, b):
        date_value = 0
        if self.date_picker != None:
            date_value = self.date_picker.value
        self.ax_1.ax.clear()
        self.ax_1.ax.grid(True)
        self.init_date = date_value
        self.n_points = self.n_slider.value
        self.draw()

In [60]:
#read-in, n = number of data points
n = int(input())
    
#runner
def main():
    parse()
    #print(mat[0][0])
    g = Graph(n, mat, 0)
    g = Graph(n, mat, 1)
    
#runner
if(__name__ == '__main__'):
    main()

10


Dropdown(description='Pick initial date:', options=(('2011-11-03', 0), ('2011-11-15', 1), ('2011-11-17', 2), (…

IntSlider(value=10, continuous_update=False, description='Number of Points:', layout=Layout(width='381px'), ma…

Button(description='Apply', style=ButtonStyle(), tooltip='Update Graph')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Dropdown(description='Pick initial date:', options=(('2010-02-19', 0), ('2010-02-24', 1), ('2010-02-26', 2), (…

IntSlider(value=10, continuous_update=False, description='Number of Points:', layout=Layout(width='381px'), ma…

Button(description='Apply', style=ButtonStyle(), tooltip='Update Graph')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …